# Vars

In [ ]:
LLM_name = "qwen2.5vl:3b"

# Load LLM

In [ ]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model=LLM_name)  

# Load embedding model

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

#test
embed = await hf.aembed_query("hello")
print(embed)

# Vector store

In [ ]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embed)
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=hf,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

print(vector_store)

# Load and chunk the data

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


loader = DirectoryLoader("./data/", glob="**/*.txt", show_progress=True)
docs = loader.load()
print("data loaded successfully")


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # chunk size (mb chars)
    chunk_overlap=200,  # chunk overlap (nb chars)
    add_start_index=True,  # track index in original document (idk wchm3ntha)
)
all_splits = text_splitter.split_documents(docs)
print(f"Split blog post into {len(all_splits)} sub-documents.")

#sotre the splits in vector sotre
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:2])

# Retrieval and Generation

**1 Create prompt**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

#create custom prompt
prompt = ChatPromptTemplate.from_template(
    "Vous êtes un assistant chargé de répondre à des questions. Utilisez les éléments de contexte suivants pour répondre à la question. Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas. Utilisez au maximum trois phrases et restez concis..\n\n"
    "Contexte : {context}\n\n"
    "Question : {question}\n\n"
    "Réponse :"
)


**2 Create the graph**

In [ ]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response}

In [ ]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

# Test (ncahlh tmchi)

In [ ]:
result = graph.invoke({"question": "Qui est le web developper?"})

print(f'Answer: {result["answer"]}')

In [ ]:
result = graph.invoke({"question": "Qui est pas le chikour? et pour quoi"})

print(f'Answer: {result["answer"]}')